In [1]:
import os
# 设置环境变量，只让程序看到 GPU 2
os.environ['CUDA_VISIBLE_DEVICES'] = '2'


import torch
import torch.nn as nn
import wandb
import random
import argparse
import numpy as np
from tqdm import tqdm
from transformers import BertModel, AutoModel
from transformers import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 数据预处理函数

In [2]:
from torch.utils.data import Dataset
import json

class BAE2025Dataset(Dataset):
    def __init__(
            self,
            data_path,
            label_type="Actionability",  # 根据需要可以是 "Mistake_Identification", "Mistake_Location", "Providing_Guidance", "Actionability"
            labels={
                "Yes": 0,
                "To some extent": 1, 
                "No": 2,
            }
    ):
        self.data_path = data_path
        self.label_type = label_type
        self.labels = labels
    #     self._get_data()

        self.data = []  # 初始化为空列表
        
        # 只有在data_path不为None时才加载数据
        if self.data_path is not None:
            self._get_data()
    
    def _get_data(self):
        with open(self.data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        self.data = []
        for item in data:
            sent1 = item['conversation_history']
            sent2 = item['response']
            
            # 检查item中是否直接包含我们需要的标签
            if self.label_type in item and item[self.label_type] in self.labels:
                self.data.append(((sent1, sent2), self.labels[item[self.label_type]]))
    
    def __len__(self):
        return len(self.data)
    
    def get_labels(self):
        return self.labels

    def __getitem__(self, idx):
        return self.data[idx]

# 数据加载函数

In [3]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import AutoConfig
from transformers import DebertaV2Tokenizer

class BAE2025DataLoader:
    def __init__(
        self,
        dataset,
        batch_size=16,
        max_length=512,
        shuffle=True,
        drop_last=True,
        device=None,
        # tokenizer_name='chinese-bert-wwm-ext'
        # tokenizer_name='chinese-roberta-wwm-ext'
        # tokenizer_name='chinese-roberta-wwm-ext-large'
        # tokenizer_name='/mnt/cfs/huangzhiwei/pykt-moekt/SBM/bge-large-en-v1.5'
        tokenizer_name='/mnt/cfs/huangzhiwei/BAE2025/models/deberta-v3-base'
        # tokenizer_name='/mnt/cfs/huangzhiwei/BAE2025/models/roberta-base'
    ):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.tokenizer.truncation_side = 'left'  # 设置截断方向为左侧,即从句子开头开始截断,假设一个句子过长，则从句子开头开始截断，保留句子结尾的部分
        print("当前使用的 tokenizer 类型：", type(self.tokenizer))
        
        # config = AutoConfig.from_pretrained(tokenizer_name)
        # self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, config=config, use_fast=True)
        
        
        # self.tokenizer = DebertaV2Tokenizer.from_pretrained(tokenizer_name)
        
        self.dataset = dataset
        self.batch_size = batch_size
        self.max_length = max_length
        self.shuffle = shuffle
        self.drop_last = drop_last

        if device is None:
            self.device = torch.device(
                'cuda' if torch.cuda.is_available() else 'cpu'
            )
        else:
            self.device = device

        self.loader = DataLoader(
            dataset=self.dataset,
            batch_size=self.batch_size,
            collate_fn=self.collate_fn,
            shuffle=self.shuffle,
            drop_last=self.drop_last
        )

    def collate_fn(self, data):
        sents = [i[0] for i in data]
        labels = [i[1] for i in data]

        # 修改这里，处理两个句子的情况
        data = self.tokenizer.batch_encode_plus(
            batch_text_or_text_pairs=[(sent[0], sent[1]) for sent in sents],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt',
            return_length=True
        )
        input_ids = data['input_ids'].to(self.device)
        attention_mask = data['attention_mask'].to(self.device)
        # token_type_ids = data['token_type_ids'].to(self.device)
        labels = torch.LongTensor(labels).to(self.device)

        # return input_ids, attention_mask, token_type_ids, labels
        return input_ids, attention_mask, labels


    def __iter__(self):
        for data in self.loader:
            yield data

    def __len__(self):
        return len(self.loader)



# 模型代码

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel


class ExpertLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
        )
        
    def forward(self, x):
        # x: [batch_size, seq_len, input_size]
        output, (hidden, _) = self.lstm(x)
        # 返回最后一个时间步的隐藏状态
        return hidden[-1]  # [batch_size, hidden_size]


class ExpertBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__()
        self.bilstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size // 2,  # 因为是双向的，所以每个方向的隐藏层大小减半
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0,
        )
        
    def forward(self, x):
        # x: [batch_size, seq_len, input_size]
        output, (hidden, _) = self.bilstm(x)
        # 拼接最后一层的正向和反向隐藏状态
        # hidden shape: [num_layers * num_directions, batch_size, hidden_size//2]
        hidden_forward = hidden[-2]  # 正向的最后一层 [batch_size, hidden_size//2]
        hidden_backward = hidden[-1]  # 反向的最后一层 [batch_size, hidden_size//2]
        hidden_concat = torch.cat([hidden_forward, hidden_backward], dim=1)  # [batch_size, hidden_size]
        return hidden_concat


class ExpertRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
        )
        
    def forward(self, x):
        # x: [batch_size, seq_len, input_size]
        _, hidden = self.rnn(x)
        # 返回最后一个时间步的隐藏状态
        return hidden[-1]  # [batch_size, hidden_size]


class ExpertGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
        )
        
    def forward(self, x):
        # x: [batch_size, seq_len, input_size]
        _, hidden = self.gru(x)
        # 返回最后一个时间步的隐藏状态
        return hidden[-1]  # [batch_size, hidden_size]


class ExpertLinear(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(input_size, hidden_size * 2),
            nn.LayerNorm(hidden_size * 2),
            nn.GELU(),
            nn.Linear(hidden_size * 2, hidden_size)
        )
        
    def forward(self, x):
        # x: [batch_size, seq_len, input_size]
        # 我们需要把序列信息压缩为一个向量，可以使用平均池化
        pooled = torch.mean(x, dim=1)  # [batch_size, input_size]
        return self.linear(pooled)  # [batch_size, hidden_size]


class BertClassificationHead(nn.Module):
    def __init__(self, hidden_size=1024, num_classes=3, dropout_prob=0.3):
        super().__init__()
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.out_proj = nn.Linear(hidden_size, num_classes)
    
    def forward(self, features):
        # 提取 [CLS] 标记的表示
        x = features[:, 0, :]  # 使用第一个标记([CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class MoERouter(nn.Module):
    """专家路由器，学习为每个样本分配专家权重"""
    def __init__(self, input_size, num_experts):
        super().__init__()
        self.router = nn.Linear(input_size, num_experts)
        
    def forward(self, x):
        # x: [batch_size, input_size]
        # 计算每个专家的权重 (使用softmax确保权重和为1)
        router_logits = self.router(x)
        router_probs = F.softmax(router_logits, dim=-1)
        return router_probs  # [batch_size, num_experts]


class DeBERTaMoEClassifier(nn.Module):
    def __init__(
        self, 
        pretrained_model_name, 
        num_classes=3, 
        freeze_pooler=0,
        expert_hidden_size=256,
        dropout=0.3,
        num_rnn_layers=1
    ):
        super().__init__()
        
        # 使用 AutoModel 加载 DeBERTa 模型
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        
        # 获取 bert 隐藏层大小
        self.bert_hidden_size = self.bert.config.hidden_size
        
        # 保留原有的分类头
        self.original_classifier = BertClassificationHead(
            hidden_size=self.bert_hidden_size,
            num_classes=num_classes,
            dropout_prob=dropout
        )
        
        # 创建多个专家模型
        self.experts = nn.ModuleDict({
            'lstm': ExpertLSTM(
                input_size=self.bert_hidden_size, 
                hidden_size=expert_hidden_size,
                num_layers=num_rnn_layers,
                dropout=dropout
            ),
            'bilstm': ExpertBiLSTM(
                input_size=self.bert_hidden_size, 
                hidden_size=expert_hidden_size,
                num_layers=num_rnn_layers,
                dropout=dropout
            ),
            'rnn': ExpertRNN(
                input_size=self.bert_hidden_size, 
                hidden_size=expert_hidden_size,
                num_layers=num_rnn_layers,
                dropout=dropout
            ),
            'gru': ExpertGRU(
                input_size=self.bert_hidden_size, 
                hidden_size=expert_hidden_size,
                num_layers=num_rnn_layers,
                dropout=dropout
            ),
            'linear': ExpertLinear(
                input_size=self.bert_hidden_size, 
                hidden_size=expert_hidden_size
            ),
        })
        
        # 创建路由器 (使用[CLS]标记表示作为路由的输入)
        self.router = MoERouter(self.bert_hidden_size, len(self.experts))
        
        # 各专家模型的输出映射层，将各自的hidden_size映射到统一的输出空间
        self.expert_outputs = nn.ModuleDict({
            expert_name: nn.Linear(expert_hidden_size, num_classes)
            for expert_name in self.experts.keys()
        })
        
        # 最终的融合层，将所有结果拼接后映射到输出类别
        # (1个原始分类头 + 5个专家) * 每个输出num_classes = 6 * num_classes
        combined_dim = num_classes * (1 + len(self.experts))
        self.final_classifier = nn.Sequential(
            nn.Linear(combined_dim, combined_dim // 2),
            nn.LayerNorm(combined_dim // 2),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(combined_dim // 2, num_classes)
        )
        
    def forward(self, input_ids, attention_mask):
        # DeBERTa 编码
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # 获取序列隐藏状态
        hidden_states = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]
        
        # 获取原始分类头结果
        original_logits = self.original_classifier(hidden_states)  # [batch_size, num_classes]
        
        # 获取路由权重
        cls_embedding = hidden_states[:, 0]  # [batch_size, hidden_size]
        routing_weights = self.router(cls_embedding)  # [batch_size, num_experts]
        
        # 获取各专家结果
        expert_outputs = {}
        for expert_name, expert in self.experts.items():
            # 获取专家输出
            expert_hidden = expert(hidden_states)  # [batch_size, expert_hidden_size]
            # 映射到类别空间
            expert_logits = self.expert_outputs[expert_name](expert_hidden)  # [batch_size, num_classes]
            # 存储结果
            expert_outputs[expert_name] = expert_logits
        
        # 根据路由权重加权专家结果
        # 首先，将所有专家的结果拼接到一起
        expert_logits_list = [original_logits]  # 包含原始分类头
        expert_names = list(self.experts.keys())
        
        for expert_name in expert_names:
            expert_logits_list.append(expert_outputs[expert_name])
        
        # 拼接所有结果 [batch_size, (1+num_experts)*num_classes]
        combined_logits = torch.cat(expert_logits_list, dim=1)
        
        # 通过最终分类器输出最终结果
        final_logits = self.final_classifier(combined_logits)
        
        # return {
        #     'logits': final_logits,  # 最终预测
        #     'original_logits': original_logits,  # 原始分类头预测
        #     'expert_logits': expert_outputs,  # 各专家预测
        #     'routing_weights': routing_weights,  # 路由权重
        #     'combined_logits': combined_logits  # 拼接的中间结果
        # }
        
        return final_logits


# FGM

In [5]:
class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}
        # 明确指定为word_embeddings
        self.emb_name = 'word_embeddings'
    
    def attack(self, epsilon=1.):
        for name, param in self.model.named_parameters():
            if param.requires_grad and self.emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)
    
    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and self.emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

# 训练参数设置

In [6]:
import os
import wandb
import random
import argparse
from tqdm import tqdm

import torch
import torch.nn as nn
import numpy as np
from transformers import AdamW
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# 如果在Jupyter Notebook中运行，可以使用这个自定义参数函数替代argparser
def get_default_configs():
    """在Jupyter环境中使用的默认配置，避免argparse解析错误"""
    class Args:
        def __init__(self):
            # self.model_name = '/mnt/cfs/huangzhiwei/pykt-moekt/SBM/bge-large-en-v1.5'
            # self.model_name = "/mnt/cfs/huangzhiwei/BAE2025/models/ModernBERT-large"
            # self.model_name = '/mnt/cfs/huangzhiwei/pykt-moekt/SBM/xlm-roberta-large'
            # self.model_name = '/mnt/cfs/huangzhiwei/BAE2025/models/bge-base-en-v1.5'
            # self.model_name = '/mnt/cfs/huangzhiwei/BAE2025/models/bert-base-uncased'
            self.model_name = '/mnt/cfs/huangzhiwei/BAE2025/models/deberta-v3-base'
            # self.model_name = '/mnt/cfs/huangzhiwei/BAE2025/models/roberta-base'
            self.num_classes = 3
            self.dropout = 0.25
            self.freeze_pooler = 8
            self.batch_size = 16
            self.max_length = 512
            self.lr = 2e-5
            self.epochs = 50
            self.device = device
            self.name = None
            self.seed = 42
            self.data_path = '../data_new/all.json'
            self.val_data_path = '../data_new/val.json'
            self.checkpoint_dir = '/mnt/cfs/huangzhiwei/BAE2025/projects/predict/5foldx'
            self.patience = 6
            self.expert_hidden_size = 512
            self.num_rnn_layers = 1
            self.warmup_ratio = 0.1
            self.exp_name = 'BAE2025_track4_bert'

            self.cross_validation = True  # 或通过命令行参数设置
            self.n_folds = 5  # 或通过命令行参数设置

    return Args()


# 训练函数

In [7]:
def train(configs, fold_idx=None):
    
    # 设置随机种子
    random.seed(configs.seed)
    np.random.seed(configs.seed)
    torch.manual_seed(configs.seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # 创建检查点目录 - 为每个折创建子目录
    # if fold_idx is not None:
    #     checkpoint_dir = os.path.join(configs.checkpoint_dir, configs.exp_name, f'fold_{fold_idx}')
    # else:
    #     checkpoint_dir = os.path.join(configs.checkpoint_dir, configs.exp_name)

    checkpoint_dir = configs.checkpoint_dir
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # 加载数据集
    if fold_idx is None:
        # 非交叉验证模式 - 使用原始的训练和验证集
        train_dataset = BAE2025Dataset(configs.data_path)
        val_dataset = BAE2025Dataset(configs.val_data_path)
    else:
        # 交叉验证模式 - 使用从全数据集分割的训练集和验证集
        # 这些数据集会在cross_validate函数中传入
        train_dataset = configs.fold_datasets[fold_idx]['train']
        val_dataset = configs.fold_datasets[fold_idx]['val']
    
    # 创建数据加载器
    train_dataloader = BAE2025DataLoader(
        dataset=train_dataset,
        batch_size=configs.batch_size,
        max_length=configs.max_length,
        shuffle=True,
        drop_last=True,
        device=configs.device,
        tokenizer_name=configs.model_name
    )

    val_dataloader = BAE2025DataLoader(
        dataset=val_dataset,
        batch_size=configs.batch_size,
        max_length=configs.max_length,
        shuffle=False,
        drop_last=False,
        device=configs.device,
        tokenizer_name=configs.model_name
    )
    
    # 创建模型
    model = DeBERTaMoEClassifier(
        pretrained_model_name=configs.model_name,
        num_classes=configs.num_classes,
        freeze_pooler=configs.freeze_pooler,
        num_rnn_layers=configs.num_rnn_layers,
        expert_hidden_size=configs.expert_hidden_size,
        dropout=configs.dropout
    ).to(configs.device)

    criterion = nn.CrossEntropyLoss()

    # 定义优化器
    optimizer = AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=configs.lr
    )
    
    # ===== 添加Warmup + Cosine Decay学习率调度 =====
    from transformers import get_cosine_schedule_with_warmup
    
    # 计算总训练步数
    total_steps = len(train_dataloader) * configs.epochs
    
    # 计算warmup步数 (默认总步数的10%，可通过configs.warmup_ratio调整)
    warmup_ratio = getattr(configs, 'warmup_ratio', 0.1)  # 如果未定义，则使用默认值0.1
    warmup_steps = int(warmup_ratio * total_steps)
    
    # 创建学习率调度器
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    # ===============================================
    
    # 初始化最佳验证损失和早停计数器
    best_val_acc = 0.0
    best_val_f1 = 0.0  # 添加F1分数作为评估指标
    best_val_loss = float('inf')
    best_epoch = -1  # 记录达到最佳效果的epoch
    patience_counter = 0
    
    # 定义类别名称
    class_names = ['Yes', 'To some extent', 'No']
    
    # 添加F1计算所需的库
    from sklearn.metrics import f1_score

    # 获得对应embedding名字
    # 打印模型中的层名称，查找嵌入层
    # for name, _ in model.named_parameters():
    #     if 'embed' in name:
    #         print(name)
    
    # # 设置FGM的嵌入层名称
    # fgm = FGM(model)
    # fgm.emb_name = 'word_embeddings'  # 根据打印结果调整正确的名称
    
    # 训练循环
    for epoch in range(configs.epochs):
        # 训练阶段
        model.train()
        train_loss = 0.0
        train_acc = 0.0
        train_preds = []
        train_labels_list = []
        
        with tqdm(
            train_dataloader,
            total=len(train_dataloader),
            desc=f'Fold {fold_idx if fold_idx is not None else "N/A"} - Epoch {epoch + 1}/{configs.epochs}',
            unit='batch',
            ncols=100
        ) as pbar:
            for input_ids, attention_mask, labels in pbar:
                optimizer.zero_grad()
                
                # 前向传播
                logits = model(input_ids, attention_mask)
                
                # 计算损失 - 确保labels是长整型
                labels = labels.long()
                loss = criterion(logits, labels)
                # 反向传播
                loss.backward()

                # # FGM对抗训练
                # fgm.attack()  # 在嵌入层添加扰动
                # # 对抗样本的前向传播
                # logits_adv = model(input_ids, attention_mask)
                # # 计算对抗损失
                # loss_adv = criterion(logits_adv, labels)
                # # 反向传播
                # loss_adv.backward()
                # # 恢复嵌入层参数
                # fgm.restore()
                
                optimizer.step()
                scheduler.step()  # 更新学习率
                
                preds = logits.argmax(dim=1)
                accuracy = (preds == labels).float().mean()
                accuracy_all = (preds == labels).float().sum()
                
                # 收集预测结果和真实标签，用于计算F1
                train_preds.extend(preds.cpu().numpy())
                train_labels_list.extend(labels.cpu().numpy())
                
                train_loss += loss.item()
                train_acc += accuracy_all.item()
                
                # 添加当前学习率到进度条
                curr_lr = scheduler.get_last_lr()[0]
                pbar.set_postfix(
                    loss=f'{loss.item():.3f}',
                    accuracy=f'{accuracy.item():.3f}',
                    lr=f'{curr_lr:.6f}'  # 显示当前学习率
                )
        
        train_loss = train_loss / len(train_dataloader)
        train_acc = train_acc / len(train_dataset)
        
        # 计算训练集的F1分数 - 使用macro平均以处理多分类
        train_f1 = f1_score(train_labels_list, train_preds, average='macro')
        
        print(f'Fold {fold_idx if fold_idx is not None else "N/A"} - Training Loss: {train_loss:.4f}')
        print(f'Fold {fold_idx if fold_idx is not None else "N/A"} - Training Accuracy: {train_acc:.4f}')
        print(f'Fold {fold_idx if fold_idx is not None else "N/A"} - Training F1 Score: {train_f1:.4f}')
        
        # 验证阶段
        model.eval()
        val_loss = 0.0
        val_corrects = 0.0
        val_preds = []
        val_labels_list = []

        with torch.no_grad():
            for input_ids, attention_mask, labels in val_dataloader:
                # 确保labels是长整型
                labels = labels.long()
                
                # 前向传播
                logits = model(input_ids, attention_mask)
                
                loss = criterion(logits, labels)
                val_loss += loss.item()
                preds = logits.argmax(dim=1)
                accuracy = (preds == labels).float().sum()
                val_corrects += accuracy
                
                # 收集预测结果和真实标签，用于计算F1
                val_preds.extend(preds.cpu().numpy())
                val_labels_list.extend(labels.cpu().numpy())
        
        val_loss = val_loss / len(val_dataloader)
        val_acc = val_corrects.double() / len(val_dataset)
        
        # 计算验证集的F1分数
        val_f1 = f1_score(val_labels_list, val_preds, average='macro')
        
        print(f'Fold {fold_idx if fold_idx is not None else "N/A"} - Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f} F1: {val_f1:.4f}')
        
        # 检查是否保存模型并判断是否需要早停
        # 使用F1分数作为主要指标
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_val_acc = val_acc
            best_epoch = epoch + 1  # 记录达到最佳效果的epoch
            
            # 保存模型 - 为每个折添加fold_idx到文件名
            if fold_idx is not None:
                model_path = os.path.join(checkpoint_dir, f'best_model_fold_{fold_idx}.pt')
            else:
                model_path = os.path.join(checkpoint_dir, 'best_model_f1.pt')
            
            state_dict = model.state_dict()
            torch.save(state_dict, model_path)
            print(f'Fold {fold_idx if fold_idx is not None else "N/A"} - New best model saved with F1: {best_val_f1:.4f}, Acc: {best_val_acc:.4f} at Epoch {best_epoch}')
            
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= configs.patience:
                print(f'Fold {fold_idx if fold_idx is not None else "N/A"} - Early stopping triggered after {epoch+1} epochs. Best model was at Epoch {best_epoch}.')
                break
    
    # 返回最佳验证指标，用于交叉验证结果汇总
    return {
        'best_val_f1': best_val_f1,
        'best_val_acc': best_val_acc,
        'best_epoch': best_epoch
    }


def cross_validate(configs, n_folds=5):
    """
    执行n折交叉验证
    
    参数:
    - configs: 配置对象
    - n_folds: 交叉验证的折数，默认为5
    """
    # 导入所需库
    from sklearn.model_selection import StratifiedKFold
    import numpy as np
    
    # 首先加载完整的数据集
    full_dataset = BAE2025Dataset(configs.data_path)
    
    # 获取标签，用于分层抽样 - 修改这一行
    labels = [data[1] for data in full_dataset.data]  # 使用.data而不是.samples
    
    # 创建分层K折交叉验证对象
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=configs.seed)
    
    # 存储每个折的数据集
    configs.fold_datasets = []
    
    # 存储每个折的结果
    fold_results = []
    
    # 遍历每个折
    for fold_idx, (train_indices, val_indices) in enumerate(skf.split(np.zeros(len(labels)), labels)):
        print(f"\n{'='*20} Fold {fold_idx+1}/{n_folds} {'='*20}")
        
        # 为当前折创建训练集和验证集 - 修改这些行
        train_samples = [full_dataset.data[i] for i in train_indices]
        val_samples = [full_dataset.data[i] for i in val_indices]
        
        # 创建新的数据集对象
        train_dataset = BAE2025Dataset(None)  # 创建空数据集
        train_dataset.data = train_samples  # 填充训练样本，使用.data而不是.samples
        
        val_dataset = BAE2025Dataset(None)  # 创建空数据集
        val_dataset.data = val_samples  # 填充验证样本，使用.data而不是.samples
        
        # 保存当前折的数据集
        configs.fold_datasets.append({
            'train': train_dataset,
            'val': val_dataset
        })
        
        # 检查标签分布是否保持一致 - 修改这些行
        train_label_dist = np.bincount(np.array([s[1] for s in train_samples])) / len(train_samples)
        val_label_dist = np.bincount(np.array([s[1] for s in val_samples])) / len(val_samples)
        full_label_dist = np.bincount(np.array(labels)) / len(labels)
        
        print(f"标签分布检查:")
        print(f"原始数据集分布: {full_label_dist}")
        print(f"训练集分布: {train_label_dist}")
        print(f"验证集分布: {val_label_dist}")
        
        # 训练当前折
        fold_result = train(configs, fold_idx=fold_idx)
        fold_results.append(fold_result)
        
        print(f"Fold {fold_idx+1} 完成，最佳F1: {fold_result['best_val_f1']:.4f}, 最佳准确率: {fold_result['best_val_acc']:.4f}")
    
    # 计算交叉验证的平均结果
    avg_f1 = sum(r['best_val_f1'] for r in fold_results) / n_folds
    avg_acc = sum(r['best_val_acc'] for r in fold_results) / n_folds
    avg_epoch = sum(r['best_epoch'] for r in fold_results) / n_folds
    
    print("\n" + "="*50)
    print(f"交叉验证完成! {n_folds}折平均结果:")
    print(f"平均最佳F1: {avg_f1:.4f}")
    print(f"平均最佳准确率: {avg_acc:.4f}")
    print(f"平均最佳Epoch: {avg_epoch:.1f}")
    print("各折结果:")
    for fold_idx, result in enumerate(fold_results):
        print(f"  Fold {fold_idx+1}: F1={result['best_val_f1']:.4f}, Acc={result['best_val_acc']:.4f}, Best Epoch={result['best_epoch']}")
    
    return fold_results


# 在以下主函数中添加判断Jupyter环境的逻辑
if __name__ == '__main__':
    # 判断是否在Jupyter环境中运行
    try:
        # 检查是否在Jupyter中运行
        get_ipython = globals().get('get_ipython', None)
        if get_ipython and 'IPKernelApp' in get_ipython().config:
            # 在Jupyter环境中运行，使用默认配置
            print("Running in Jupyter environment, using default configs")
            configs = get_default_configs()
        else:
            # 在命令行环境中运行，使用argparse
            configs = argparser()
    except:
        # 任何异常都使用argparse处理
        configs = argparser()
    
    # 设置实验名称
    if configs.name is None:
        configs.exp_name = \
            f'{os.path.basename(configs.model_name)}' + \
            f'{"_fp" if configs.freeze_pooler else ""}' + \
            f'_b{configs.batch_size}_e{configs.epochs}' + \
            f'_len{configs.max_length}_lr{configs.lr}'
    else:
        configs.exp_name = configs.name
    
    # 设置设备
    if configs.device is None:
        configs.device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu'
        )
    
    # 判断是否使用交叉验证
    use_cv = getattr(configs, 'cross_validation', False)
    n_folds = getattr(configs, 'n_folds', 5)
    
    if use_cv:
        # 执行交叉验证
        print(f"执行{n_folds}折交叉验证...")
        cross_validate(configs, n_folds=n_folds)
    else:
        # 执行常规训练
        print("执行常规训练...")
        train(configs)

Running in Jupyter environment, using default configs
执行5折交叉验证...

==================== Fold 1/5 ====================
标签分布检查:
原始数据集分布: [0.52907916 0.14903069 0.32189015]
训练集分布: [0.52929293 0.1489899  0.32171717]
验证集分布: [0.52822581 0.14919355 0.32258065]


/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


当前使用的 tokenizer 类型： <class 'transformers.models.deberta_v2.tokenization_deberta_v2_fast.DebertaV2TokenizerFast'>
当前使用的 tokenizer 类型： <class 'transformers.models.deberta_v2.tokenization_deberta_v2_fast.DebertaV2TokenizerFast'>


/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Fold 0 - Epoch 1/50: 100%|█| 123/123 [00:49<00:00,  2.47batch/s, accuracy=0.500, loss=1.047, lr=0.00


Fold 0 - Training Loss: 1.0948
Fold 0 - Training Accuracy: 0.3606
Fold 0 - Training F1 Score: 0.3155
Fold 0 - Validation Loss: 0.9907 Acc: 0.5262 F1: 0.2471
Fold 0 - New best model saved with F1: 0.2471, Acc: 0.5262 at Epoch 1


...

Fold 4 - Training Loss: 0.7485
Fold 4 - Training Accuracy: 0.7456
Fold 4 - Training F1 Score: 0.5412
Fold 4 - Validation Loss: 0.7821 Acc: 0.7232 F1: 0.5167


Fold 4 - Epoch 22/50: 100%|█| 123/123 [00:49<00:00,  2.50batch/s, accuracy=0.625, loss=0.941, lr=0.0


Fold 4 - Training Loss: 0.7481
Fold 4 - Training Accuracy: 0.7587
Fold 4 - Training F1 Score: 0.5583
Fold 4 - Validation Loss: 0.7740 Acc: 0.7313 F1: 0.5236


Fold 4 - Epoch 23/50: 100%|█| 123/123 [00:49<00:00,  2.50batch/s, accuracy=0.812, loss=0.641, lr=0.0


Fold 4 - Training Loss: 0.7501
Fold 4 - Training Accuracy: 0.7481
Fold 4 - Training F1 Score: 0.5478
Fold 4 - Validation Loss: 0.7707 Acc: 0.7354 F1: 0.5257
Fold 4 - Early stopping triggered after 23 epochs. Best model was at Epoch 17.
Fold 5 完成，最佳F1: 0.5274, 最佳准确率: 0.7354

交叉验证完成! 5折平均结果:
平均最佳F1: 0.5896
平均最佳准确率: 0.7250
平均最佳Epoch: 20.8
各折结果:
  Fold 1: F1=0.6174, Acc=0.6915, Best Epoch=27
  Fold 2: F1=0.5386, Acc=0.7495, Best Epoch=18
  Fold 3: F1=0.6341, Acc=0.7374, Best Epoch=20
  Fold 4: F1=0.6306, Acc=0.7111, Best Epoch=22
  Fold 5: F1=0.5274, Acc=0.7354, Best Epoch=17
